Используя Bokeh, постройте  круговую диаграмму - количество вакансий внутри города. Выдвиньте клин, соответствующий городу Москва. Добавьте название диаграммы. В качестве источника данных используйте файл по ссылке ниже:

https://disk.yandex.ru/i/yDppoT1Txpy7Fg